In [1]:
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
from tqdm import tqdm
import logging
from zenmoney import Request, Diff
from dotenv import dotenv_values

env = dotenv_values()

# Firefly cleanup & API

## API config

In [2]:
api_url = f"{env['FIREFLY_URL']}/api/v1"

headers = {
    'accept': 'application/vnd.api+json',
    'Authorization': f"Bearer {env['FIREFLY_TOKEN']}",
    'Content-Type': 'application/json'
}

## Cleanup

In [220]:
requests.delete(api_url + f'/data/destroy', json={'objects': 'accounts'}, headers=headers)
requests.delete(api_url + f'/data/destroy', json={'objects': 'liabilities'}, headers=headers)
requests.delete(api_url + f'/data/destroy', json={'objects': 'transactions'}, headers=headers)

<Response [204]>

```sql
delete from firefly.accounts where TRUE;
delete from firefly.transactions where TRUE;
delete from firefly.transaction_currencies where deleted_at is not null;
```

# Zenmoney API

In [221]:
class ZenmoneyCollector:
    def __init__(self, api_token) -> None:

        zenmoney_api = Request(api_token)
        self.zenmoney_diff = zenmoney_api.diff(Diff(**{"serverTimestamp": 0})).to_dict()

        self.zen_types_processing = {
            'account': {
                'changed': self.dt_from_unix_time,
                'startDate': self.dt_from_date_string
            },
            'transaction': {
                'date': self.dt_from_date_string,
                'changed': self.dt_from_unix_time,
                'created': self.dt_from_unix_time,
                'tag': lambda x: x[0] if isinstance(x, list) else None
            },
            'merchant': {
                'changed': self.dt_from_unix_time
            },
            'user': {
                'changed': self.dt_from_unix_time
            },
            'tag': {
                'changed': self.dt_from_unix_time,
                'color': self.rgb 
            },
            'instrument': {
                'changed': self.dt_from_unix_time
            }
        }

    def dt_from_unix_time(self,value):
        if isinstance(value, int):
            return datetime.fromtimestamp(value)
        else:
            return value
        
    def dt_from_date_string(self, value):
        if value == "None":
            return None
        elif isinstance(value, str):
            return datetime.strptime(value, "%Y-%m-%d")
        else:
            return value
        
    def rgb(self, int_color: int):
        if isinstance(int_color, int):
            a = (int_color >> 24) & 0x000000FF
            r = (int_color >> 16) & 0x000000FF
            g = (int_color >> 8) & 0x000000FF
            b = (int_color >> 0) & 0x000000FF
        else:
            a, r, g, b = (0 ,0, 0, 0)
        
        return "#{:02x}{:02x}{:02x}".format(r, g, b)
    
    def load_objects(self, *objects):
        zen_objects = {}
        for object_ in objects:
            zo = self.zenmoney_diff[object_].copy()

            for object_instance in zo:
                for field in object_instance.keys():
                    if field in self.zen_types_processing[object_].keys():
                        object_instance[field] = self.zen_types_processing[object_][field](object_instance[field])

            zo = {i['id']: i for i in zo}

            if object_ in ['transaction']:
                zo = {d[0]: d[1] for d in zo.items() if not d[1]['deleted']}

            zen_objects[object_] = zo

        return zen_objects

# Currencies

In [211]:
with open('currency-format.json', 'r') as file:
    currencies = json.load(file)

def currency_cleanup():
    ff_currencies = requests.get(api_url + f'/currencies', headers=headers).json()['data']
    ff_currencies = [i['attributes']['code'] for i in ff_currencies]
    for c in ff_currencies:
        response = requests.delete(api_url + f"/currencies/{c}", headers=headers)
        if not response:
            print(response.json())

def currency_mapper(currencies, replace=True, default='EUR', enabled=[]):
    ff_currencies = requests.get(api_url + f'/currencies', headers=headers).json()['data']
    ff_currencies = [i['attributes']['code'] for i in ff_currencies]
    
    for code, attrs in currencies.items():
        
        new_entry = {
            "decimal_places": attrs['fractionSize'],
            'enabled': (code in enabled),
            'default': code==default
        }

        if code in ff_currencies:
            if replace:
                response = requests.put(api_url + f'/currencies/{code}', json=new_entry, headers=headers)
            else:
                continue
        else:
            new_entry['code'] = code
            new_entry['name'] = attrs['name']
            new_entry['symbol'] = attrs['uniqSymbol']['grapheme'] if attrs['uniqSymbol'] else code
            response = requests.post(api_url + f'/currencies', json=new_entry, headers=headers)
        
        if not response:    
            print(code, response.json())

In [ ]:
#currency_cleanup()

In [ ]:
currency_mapper(currencies, replace=True, enabled=['USD', 'EUR', 'AMD', 'RUB', 'HKD', 'LKR', 'THB', 'TON'])

# Import

In [222]:
class Zenmoney2Firefly:

    def __init__(self, zenmoney_token) -> None:
        self.__zenmoney_token = zenmoney_token
        self.__load_zen_objects()

        self.debt_id = list(filter(lambda x: x['title'] == 'Долги', self.zen_objects['account'].values()))
        assert len(self.debt_id) == 1
        self.debt_id = self.debt_id[0]['id']

        self.deduplicate_account_names()
        self.set_opening_balance_dates()
        self.rename_debt_payees()

        self.entity_import_func_mapping = {
            'asset_accounts': self.asset_account_mapper,
            'revenue_accounts': self.revenue_account_mapper,
            'liability_accounts': self.liability_account_mapper,
            'transactions': self.transactions_mapper,
        }

    def import_entities(self, entities=None, map_debt_expense=True):
        if not entities:
            entities = list(self.entity_import_func_mapping.keys())
        if map_debt_expense:
            self.map_zenmoney_debt_to_expenses()

        for k in entities:
            self.entity_import_func_mapping[k]()

    def __load_zen_objects(self):
        objects =  ['instrument', 'account', 'tag', 'merchant', 'transaction']
        self.zen_objects = ZenmoneyCollector(self.__zenmoney_token).load_objects(*objects)
        self.zen_objects_initial = self.zen_objects.copy()

    def deduplicate_account_names(self):
        names, names_count = np.unique([d['title'] for _, d in self.zen_objects['account'].items()], return_counts=1)
        duplicate_names = list(names[names_count > 1])

        for duplicate_name in duplicate_names:
            i = 1
            for k, v in self.zen_objects['account'].items():
                name = v['title']
                if name == duplicate_name:
                    self.zen_objects['account'][k]['title'] += f' ({i})'
                    i += 1

    def first_transaction_date(self, account_id):
        acc_transactions = list(filter(lambda x: x['outcomeAccount']==account_id or x['incomeAccount']==account_id, self.zen_objects['transaction'].values()))
        acc_transactions = [i['date'] for i in acc_transactions]

        if len(acc_transactions) > 0:
            ftd = min(acc_transactions)
            return ftd
        else:
            return None

    def set_opening_balance_dates(self):
        for id in self.zen_objects['account'].keys():
            ftd = self.first_transaction_date(id)

            if ftd is not None:
                obd = (ftd - timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
            else:
                obd = self.zen_objects['account'][id]['changed'].strftime('%Y-%m-%d %H:%M:%S')
            
            self.zen_objects['account'][id]['opening_balance_date'] = obd

    def rename_debt_payees(self):
        for id in self.zen_objects['transaction'].keys():
            transaction = self.zen_objects['transaction'][id]

            if transaction['incomeAccount'] == self.debt_id or transaction['outcomeAccount'] == self.debt_id:
                merchant = transaction['merchant']
                payee = transaction['payee']

                if merchant is not None:
                    merchant_name = self.zen_objects['merchant'][merchant]['title'].strip().strip('"')
                elif payee is not None:
                    merchant_name = payee.strip().strip('"')
                else:
                    merchant_name = 'Unknown'
        
                currency = transaction['outcomeInstrument']
                currency = self.zen_objects['instrument'][currency]['shortTitle']

                self.zen_objects['transaction'][id]['liability_payee_name'] = f"{merchant_name} | {currency}"
                self.zen_objects['transaction'][id]['liability_currency'] = currency

    def post_entity(self, entity_name, entity_dict):
        response = requests.post(api_url + f'/{entity_name}', json=entity_dict, headers=headers)

        if not response:
            print(response.json())
        else:
            return response.json()

    def nvl(self, *args):
        for arg in args:
            if arg:
                return arg
            
        return 



    '''
    Currencies
    '''



    '''
    Asset accounts
    '''

    def asset_account_cleanup(self):
        ff_accounts = requests.get(api_url + f'/accounts', headers=headers).json()['data']
        for account in ff_accounts:
            if account['attributes']['type'] == 'asset':
                response = requests.delete(api_url + f"/accounts/{account['id']}", headers=headers)
   
    def asset_account_mapper(self):
        zen_ff_atm = {
            'checking': ('asset', 'defaultAsset'),
            'deposit': ('asset', 'defaultAsset'),
            'ccard': ('asset', 'ccAsset'),
            'cash': ('asset', 'cashWalletAsset'),
        }

        for id, account in self.zen_objects['account'].items():
            if account['type'] != 'debt':
                new_entry = {
                    "name": account['title'],
                    "type": zen_ff_atm[account['type']][0],
                    "account_role": zen_ff_atm[account['type']][1] if not account['savings'] else 'savingAsset',
                    'opening_balance': account['startBalance'],
                    'opening_balance_date': account['opening_balance_date'],
                    'virtual_balance': account['creditLimit'],
                    'currency_code': self.zen_objects['instrument'][account['instrument']]['shortTitle'],
                    'active': not account['archive'],
                    'include_net_worth': account['inBalance']
                    }
                
                if account['type']=='ccard':
                    new_entry['credit_card_type'] = 'monthlyFull'
                    new_entry['monthly_payment_date'] = account['changed'].strftime('%Y-%m-%d %H:%M:%S')
                
                response = self.post_entity('accounts', new_entry)
                if response:
                    self.zen_objects['account'][id]['firefly_id'] = response['data']['id']

    def asset_account_checker(self, zen_account_id):
        if zen_account_id == self.debt_id:
            return
        
        try:
            zen_account = self.zen_objects['account'][zen_account_id]
        except KeyError:
            raise ValueError(f'There is no Zenmoney account with ID {zen_account_id}')
        
        try:
            firefly_account_id = zen_account['firefly_id']
        except KeyError:
            raise ImportError(f"There is no Firefly III account created for Zenmoney account \"{zen_account['title']}\"")
        
        firefly_account = requests.get(api_url + f'/accounts/{firefly_account_id}', headers=headers)
        if firefly_account:
            firefly_account = firefly_account.json()['data']['attributes']
        else:
            raise ConnectionError('Unable to retrieve Firefly account info')
        
        firefly_balance = float(firefly_account['current_balance']) - float(firefly_account['virtual_balance'])
        zen_balance = zen_account['balance']
        zen_start_value = zen_account['startBalance']
        
        if not np.isclose(firefly_balance, zen_balance, atol=1e-2, rtol=0):
            print(f"Balance incostistency for account \"{zen_account['title']}\"")

            if np.isclose(firefly_balance - zen_balance, zen_start_value, atol=1e-2, rtol=0):
                print(f"\tFixing balance incostistency via start value for account \"{zen_account['title']}\"")

                obt = requests.get(api_url + '/transactions', headers=headers, params={'type': 'opening_balance', 'limit': '1000'}).json()['data']
                obt = list(filter(lambda x: x['attributes']['transactions'][0]['destination_id'] == firefly_account_id, obt))[0]
                obt_id = obt['id']

                requests.delete(api_url + f'/transactions/{obt_id}', headers=headers)
            else:
                print(f"\tCannot fix balance incostistency for account \"{zen_account['title']}\"")
        else:
            print(f"Balance OK for account \"{zen_account['title']}\"")



    '''
    Revenue accounts
    '''

    def revenue_account_cleanup(self):
        ff_accounts = requests.get(api_url + f'/accounts', headers=headers).json()['data']
        for account in ff_accounts:
            if account['attributes']['type'] == 'revenue':
                response = requests.delete(api_url + f"/accounts/{account['id']}", headers=headers)

    def revenue_builder(self):
        zen_income_transactions = [i for i in self.zen_objects['transaction'].values() if i['incomeAccount'] != self.debt_id and i['outcomeAccount'] != self.debt_id and i['income'] > 0 and i['outcome'] == 0]
        
        zen_income_merchants = set([i['merchant'] for i in zen_income_transactions])
        zen_income_merchants = [i['title'] for i in self.zen_objects['merchant'].values() if i['id'] in zen_income_merchants]
        zen_income_merchants = set(zen_income_merchants)

        zen_income_payees = set([i['payee'] for i in zen_income_transactions])

        income_entities = list(zen_income_merchants.union(zen_income_payees))

        return income_entities
                
    def revenue_account_mapper(self):
        revenue_entries = self.revenue_builder()

        for title in revenue_entries:
            new_entry = {
                "name": title if title else 'Unknown',
                "type": 'revenue',
                }
            
            _ = self.post_entity('accounts', new_entry)



    '''
    Liability accounts
    '''

    def isnull(self, v, fallback):
        if v is not None:
            return v
        else:
            return fallback

    def liability_account_cleanup(self):
        ff_accounts = requests.get(api_url + f'/accounts', headers=headers).json()['data']
        for account in ff_accounts:
            if account['attributes']['type'] == 'liabilities':
                response = requests.delete(api_url + f"/accounts/{account['id']}", headers=headers)


    def liability_builder(self):
        zen_debt_transactions = [i for i in self.zen_objects['transaction'].values() if i['incomeAccount'] == self.debt_id or i['outcomeAccount'] == self.debt_id]
        liability_entities = set([(i['liability_payee_name'], i['liability_currency']) for i in zen_debt_transactions])
        liability_entities = set(liability_entities)

        return liability_entities

    def liability_account_mapper(self):
        liability_entries = self.liability_builder()

        for name, currency in liability_entries:
            new_entry = {
                "name": name,
                "type": 'liability',
                'liability_type': 'debt',
                'liability_direction': 'credit',
                'currency_code': currency
                }
            
            _ = self.post_entity('accounts', new_entry)



    '''
    Transactions
    '''

    def transaction_converter(self, t):

        if t['outcome'] > 0 and t['income'] == 0 and t['incomeAccount'] != self.debt_id and t['outcomeAccount'] != self.debt_id:
            transaction_type = 'withdrawal'
        elif t['outcome'] > 0 and t['income'] > 0 and t['incomeAccount'] != self.debt_id and t['outcomeAccount'] != self.debt_id:
            transaction_type = 'transfer'
        elif (t['incomeAccount'] == self.debt_id or t['outcomeAccount'] == self.debt_id):
            transaction_type = 'liability'
        elif t['outcome'] == 0 and t['income'] > 0:
            transaction_type = 'deposit'
        else:
            raise ValueError('Unknown transaction specification.')
        
        if transaction_type == 'withdrawal':
            new_entry = {
                'apply_rules': False, 
                'fire_webhooks': False,
                'transactions': [{
                    'amount': t['outcome'],
                    'date': t['date'].strftime('%Y-%m-%d'),
                    'description': t['comment'] if t['comment'] else '-',
                    'type': 'withdrawal',
                    'currency_code': self.zen_objects['instrument'][t['outcomeInstrument']]['shortTitle'],
                    'category_name': self.zen_objects['tag'][t['tag']]['title'] if t['tag'] else 'Unknown',
                    'source_name': self.zen_objects['account'][t['outcomeAccount']]['title'],
                    'destination_name': self.nvl(self.zen_objects['merchant'].get(t['merchant'], {'title': None})['title'], t['payee'], 'Undefined'),
                    'external_id': t['id'],
                    'process_date': t['created'].strftime('%Y-%m-%d')
                }]
            }

        if transaction_type == 'deposit': 
            new_entry = {
                'apply_rules': False, 
                'fire_webhooks': False,
                'transactions': [{
                    'amount': t['income'],
                    'date': t['date'].strftime('%Y-%m-%d'),
                    'description': t['comment'] if t['comment'] else '-',
                    'type': 'deposit',
                    'currency_code': self.zen_objects['instrument'][t['incomeInstrument']]['shortTitle'],
                    'category_name': self.zen_objects['tag'][t['tag']]['title'] if t['tag'] else 'Unknown',
                    'source_name': self.nvl(self.zen_objects['merchant'].get(t['merchant'], {'title': None})['title'], t['payee'], 'Undefined'),
                    'destination_name': self.zen_objects['account'][t['incomeAccount']]['title'],
                    'external_id': t['id'],
                    'process_date': t['created'].strftime('%Y-%m-%d')
                }]
            }

        if transaction_type == 'transfer':
            new_entry = {
                'apply_rules': False, 
                'fire_webhooks': False,
                'transactions': [{
                    'amount': t['outcome'],
                    'date': t['date'].strftime('%Y-%m-%d'),
                    'description': t['comment'] if t['comment'] else '-',
                    'type': 'transfer',
                    'currency_code': self.zen_objects['instrument'][t['outcomeInstrument']]['shortTitle'],
                    'source_name': self.zen_objects['account'][t['outcomeAccount']]['title'],
                    'destination_name': self.zen_objects['account'][t['incomeAccount']]['title'],
                    'foreign_amount': t['income'],
                    'foreign_currency_code': self.zen_objects['instrument'][t['incomeInstrument']]['shortTitle'],
                    'external_id': t['id'],
                    'process_date': t['created'].strftime('%Y-%m-%d')
                }]
            }

        if transaction_type == 'liability':
            ttype = 'withdrawal'

            source_name = self.zen_objects['account'][t['outcomeAccount']]['title']
            if source_name == 'Долги':
                source_name = t['liability_payee_name']
                ttype = 'deposit'
            
            destination_name = self.zen_objects['account'][t['incomeAccount']]['title']
            if destination_name == 'Долги':
                destination_name = t['liability_payee_name']
                #ttype = 'deposit'

            new_entry = {
                'apply_rules': False, 
                'fire_webhooks': False,
                'transactions': [{
                    'amount': t['outcome'],
                    'date': t['date'].strftime('%Y-%m-%d'),
                    'description': t['comment'] if t['comment'] else '-',
                    'type': ttype,
                    'currency_code': self.zen_objects['instrument'][t['outcomeInstrument']]['shortTitle'],
                    'source_name': source_name,
                    'destination_name': destination_name,
                    # 'foreign_amount': t['income'],
                    # 'foreign_currency_code': self.zen_objects['instrument'][t['incomeInstrument']]['shortTitle'],
                    'external_id': t['id'],
                    'process_date': t['created'].strftime('%Y-%m-%d')
                }]
            }
        
        return new_entry

    def transactions_mapper(self):
        for id, transaction in tqdm(self.zen_objects['transaction'].items()):
            if transaction.get('is_mapped', False):
                new_entry = self.mapped_transaction_converter(id)
                if new_entry is None:
                    continue
            else:
                new_entry = self.transaction_converter(transaction)
            
            _ = self.post_entity('transactions', new_entry)

    '''
    Debt and Expense Mapping
    '''

    def map_zenmoney_debt_to_expenses(self):
        zen_transactions_df = pd.DataFrame.from_dict(self.zen_objects['transaction'], orient='index')
        zen_transactions_df = zen_transactions_df[~zen_transactions_df.deleted]

        zen_transactions_df['is_expense'] = (
            (zen_transactions_df['outcome'] > 0) & \
            (zen_transactions_df['income'] == 0) & \
            (zen_transactions_df['incomeAccount'] != self.debt_id) & \
            (zen_transactions_df['outcomeAccount'] != self.debt_id)
            )

        zen_transactions_df['is_debt_sponsor'] = (zen_transactions_df['outcomeAccount'] == self.debt_id)
        zen_transactions_df['is_debt_payment'] = (zen_transactions_df['incomeAccount'] == self.debt_id)

        zen_transactions_df = zen_transactions_df[zen_transactions_df.is_expense | zen_transactions_df.is_debt_sponsor | zen_transactions_df.is_debt_payment]

        zen_transactions_df['mapping_account']  = np.where(zen_transactions_df['is_expense'] | zen_transactions_df['is_debt_payment'], zen_transactions_df['outcomeAccount'], zen_transactions_df['incomeAccount'])
        zen_transactions_df['mapping_value']  = np.where(zen_transactions_df['is_expense'] | zen_transactions_df['is_debt_payment'], zen_transactions_df['outcome'], zen_transactions_df['income'])

        zen_transactions_df['mapping_merchant']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['merchant'], pd.NA)
        zen_transactions_df['mapping_merchant_second']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['payee'], pd.NA)
        zen_transactions_df['mapping_payee']  = np.where(zen_transactions_df['is_debt_sponsor'], zen_transactions_df['payee'], pd.NA)

        zen_transactions_df['description']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['comment'], pd.NA)
        zen_transactions_df['currency']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['outcomeInstrument'], pd.NA)
        zen_transactions_df['tag']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['tag'], pd.NA)

        zen_transactions_df['mapping_debt_transaction_id']  = np.where(zen_transactions_df['is_debt_sponsor'], zen_transactions_df['id'], pd.NA)
        zen_transactions_df['mapping_expense_transaction_id']  = np.where(zen_transactions_df['is_expense'], zen_transactions_df['id'], pd.NA)


        def any_notnull(x):
            x_nn = [i for i in x.values if not pd.isna(i)]
            try:
                return x_nn[0]
            except IndexError:
                return np.nan

        zen_mapping = zen_transactions_df.groupby(['date', 'mapping_account', 'mapping_value']).agg({
            'id': 'count',
            'is_debt_sponsor': 'sum',
            'is_debt_payment': 'sum',
            'is_expense': 'sum',
            'created': 'min',
            'currency': any_notnull,
            'description': any_notnull,
            'mapping_payee': any_notnull, 
            'mapping_merchant': any_notnull,
            'mapping_merchant_second': any_notnull,
            'mapping_debt_transaction_id': any_notnull,
            'mapping_expense_transaction_id': any_notnull,
            'tag': any_notnull
        })

        zen_mapping = zen_mapping[
            (zen_mapping['id'] ==  2) & \
            (zen_mapping['is_debt_sponsor'] ==  1) & \
            (zen_mapping['is_debt_payment'] ==  0) & \
            (zen_mapping['is_expense'] ==  1)
            ]

        zen_mapping['destination_name'] = zen_mapping.apply(lambda x: self.nvl(self.zen_objects['merchant'].get(x['mapping_merchant'], {'title': None})['title'], x['mapping_merchant_second'], 'Undefined'), axis=1)
        zen_mapping['currency_code'] = zen_mapping.apply(lambda x: self.zen_objects['instrument'][x['currency']]['shortTitle'], axis=1)
        zen_mapping['category'] = zen_mapping.apply(lambda x: self.zen_objects['tag'][x['tag']]['title'] if x['tag'] else 'Unknown', axis=1)

        zen_mapping = zen_mapping.fillna({'description': '-', 'destination_name': 'Undefined'})

        mapped_debt_transactions = zen_mapping['mapping_debt_transaction_id'].unique()
        mapped_expense_transactions = zen_mapping['mapping_expense_transaction_id'].unique()

        for t in self.zen_objects['transaction'].keys():
            if (t in mapped_debt_transactions) or (t in mapped_expense_transactions):
                self.zen_objects['transaction'][t]['is_mapped'] = True

        self.zen_mapping = zen_mapping

    def mapped_transaction_converter(self, id):
        t = self.zen_mapping[self.zen_mapping.mapping_debt_transaction_id == id]
        n = t.shape[0]

        if n == 0:
            return
        elif n == 1:
            t = t.reset_index().iloc[0]
            new_entry = {
                'apply_rules': False, 
                'fire_webhooks': False,
                'transactions': [{
                    'amount': t['mapping_value'],
                    'date': t['date'].strftime('%Y-%m-%d'),
                    'description': t['description'] if t['description'] else '-',
                    'type': 'withdrawal',
                    'currency_code': t['currency_code'],
                    'category_name': t['category'],
                    'source_name': self.zen_objects['transaction'][id]['liability_payee_name'],
                    'destination_name': t['destination_name'],
                    'external_id': id,
                    'process_date': t['created'].strftime('%Y-%m-%d')
                }]
            }
            return new_entry
        else:
            raise ValueError('Some shit going on....')

In [223]:
z2f = Zenmoney2Firefly(env['ZENMONEY_TOKEN'])

In [ ]:
z2f.import_entities()

# Import check

In [ ]:
for id in z2f.zen_objects['account'].keys():
    try:
        z2f.asset_account_checker(id)
    except Exception as e:
        print(e)